In [ ]:
%cd /content/
!rm -rf cloth-segmentation
!git clone https://github.com/levindabhi/cloth-segmentation.git
%cd cloth-segmentation
!gdown --id 1mhF3yqd7R-Uje092eypktNl-RoZNuiCJ
!mkdir input_images
!mkdir output_images

In [ ]:
import os
# from tqdm import tqdm
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from data.base_dataset import Normalize_image
from utils.saving_utils import load_checkpoint_mgpu

from networks import U2NET
device = 'cuda'

image_dir = 'path image directory'
result_dir = 'path result directory'
checkpoint_path = 'path checkpoint'

def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


transforms_list = []
transforms_list += [transforms.ToTensor()]
transforms_list += [Normalize_image(0.5, 0.5)]
transform_rgb = transforms.Compose(transforms_list)

net = U2NET(in_ch=3, out_ch=4)
net = load_checkpoint_mgpu(net, checkpoint_path)
net = net.to(device)
net = net.eval()

palette = get_palette(4)

!rm -rf input_images/.ipynb_checkpoints
images_list = sorted(os.listdir(image_dir))
pbar = tqdm(total=len(images_list))
outf = sorted(os.listdir(result_dir))


for image_name in images_list:

      if "aug" not in image_name:
        if image_name.endswith("jpg"):
          if image_name[:-4]+'_generated.png' not in outf :
            img = Image.open(os.path.join(image_dir, image_name)).convert('RGB')
            img_size = img.size
            
            img = img.resize((768, 768), Image.BICUBIC)
            image_tensor = transform_rgb(img)
            image_tensor = torch.unsqueeze(image_tensor, 0)
            
            output_tensor = net(image_tensor.to(device))
            output_tensor = F.log_softmax(output_tensor[0], dim=1)
            output_tensor = torch.max(output_tensor, dim=1, keepdim=True)[1]
            output_tensor = torch.squeeze(output_tensor, dim=0)
            output_tensor = torch.squeeze(output_tensor, dim=0)
            output_arr = output_tensor.cpu().numpy()

            output_img = Image.fromarray(output_arr.astype('uint8'), mode='L')
            output_img = output_img.resize(img_size, Image.BICUBIC)
            
            output_img.putpalette(palette)
            output_img.save(os.path.join(result_dir, image_name[:-4]+'_generated.png'))
            
            pbar.update(1)

pbar.close()

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt



image_dir = 'path segmentation directory'

orginal_dir = 'path IMAGES directory'


result_dir = 'path result directory'

images = sorted(os.listdir(image_dir))
n = 1
for img_file in images:

		image = cv2.imread(os.path.join(image_dir, img_file))
		
		orginal_img = img_file[:-14]+".jpg"
		img = cv2.imread(os.path.join(orginal_dir , orginal_img))
		
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		blurred = cv2.GaussianBlur(gray, (7, 7), 0)
		(T, threshInv) = cv2.threshold(blurred, 10, 255,
			cv2.THRESH_BINARY)
		masked = cv2.bitwise_and(img, img, mask=threshInv)
		n+=1
		cv2.imwrite(os.path.join(result_dir , orginal_img) ,masked)
		if n%5 == 0 :
			print(f'{n} images done')